## Import

In [5]:
from newspaper import Article, ArticleException
import nltk
import os
import time
from utils_art import *
import hashlib
from dateparser import parse as parse_date
import pandas as pd
from datetime import date, datetime, timedelta
import text_analysis
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings('ignore')
ts = text_analysis.text_analysis()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Functions

In [6]:
def urlToDict(url, selection=0) :
    article = urlToArticle(url)
    if type(article) != type(None) :
        out_dict = articleToDict(article, selection)
        return out_dict
    else :
        return None

def urlToArticle(url, display=False,nlp=False) :
    article = Article(url)
    try:
        article.download()
        article.parse()
        if nlp :
            article.nlp()
        return article
    except ArticleException:
        if display :
            print("WARNING : url could not parse :",url)
        pass
        return None
    except KeyboardInterrupt:
        pass
        return None

def articleToDict(article, selection=0) :
    list_parse_main = []
    list_parse_total = ["html", "title", "authors", "publish_date", "text", "top_image", "images", "movies", "keywords", "summary"]
    list_parse_simple = ["title", "authors", "publish_date", "text", "keywords", "summary"]
    list_parse_stats = ["title", "authors", "publish_date", "keywords"]
    list_parse_stats_text = ["title", "authors", "publish_date", "keywords","summary", "text"]
    # list_parse_total = ["title", "publish_date", "text", "keywords", "summary"]
    out_dict = {}
    if selection == 0 :
        list_parse_main = list_parse_total
    if selection == 1 :
        list_parse_main = list_parse_simple
    if selection == 2 :
        list_parse_main = list_parse_stats
    if selection == 3 :
        list_parse_main = list_parse_stats_text
    for parse_str in list_parse_main :
        if parse_str == "html" :
            out_dict[parse_str] = article.html
        if parse_str == "title" :
            out_dict[parse_str] = article.title
        if parse_str == "authors" :
            out_dict[parse_str] = article.authors
        if parse_str == "publish_date" :
            out_dict[parse_str] = article.publish_date
        if parse_str == "text" :
            out_dict[parse_str] = article.text
        if parse_str == "top_image" :
            out_dict[parse_str] = article.top_image
        if parse_str == "images" :
            out_dict[parse_str] = article.images
        if parse_str == "movies" :
            out_dict[parse_str] = article.movies
        if parse_str == "keywords" :
            out_dict[parse_str] = article.keywords
        if parse_str == "summary" :
            out_dict[parse_str] = article.summary
    return out_dict

def getStandardDfInput():
    return pd.DataFrame([], columns = input_fields) 

def getStandardDfOutput():
    return pd.DataFrame([], columns = output_fields) 

def getStartArticle(article_dict,length=300) :
    text = article_dict["text"]
    if len(text) > length :
        return text[0:length]
    else :
        return text[0:len(text)]
    
# def readArticleList(url_list) :
#     start_list = []
#     count = 0
#     for url in url_list :
#         ar_dict = urlToDict(url,1)
#         if type(ar_dict) != type(None) :
#             ar_sta = getStartArticle(ar_dict)
#             start_list.append(ar_sta)
#         count = count + 1
#     return start_list

def readStatsFromURL(url, saveArticle=False, display=False,increment=0,add_nlp=False) :
    valid = True
    pk = url.split("articles/")[1].replace("?oc=5","")
    hash_key = hashlib.shake_256(str(pk).encode()).hexdigest(20)
    out_dict = {"url":url,"pk":pk,"hash_key":hash_key}
    ar_dict = urlToDict(url,3)
    display_text = " - Loadind Article #"+str(increment)+"   ("
    if type(ar_dict) != type(None) :
        text = ar_dict['text']
        text_len = len(text)
        if text_len != 0:
            if type(ar_dict['publish_date']) == type(None) :
                publish_date = ""
            else :
                if type(parse_date(str(ar_dict['publish_date']))) == type(None) :
                    publish_date = ""
                else :
                    publish_date = str(parse_date(str(ar_dict['publish_date'])).date().strftime('%Y-%m-%d'))
            analysis_nlp_dict = {}
            if add_nlp == 2:
                analysis_nlp_dict = ts.analyseArticle(text)
            if add_nlp == 1:
                analysis_nlp_dict = ts.lenStats(text)
            out_dict = out_dict | ar_dict
            out_dict["publish_date"] = publish_date
            out_dict["valid"] = True
            out_dict["text_len"] = text_len
            out_dict["keywords_list"] = out_dict["keywords"]
            out_dict["summary"] = out_dict["summary"]
            out_dict = out_dict | analysis_nlp_dict
            if saveArticle :
                dict_for_save = {"hash_key":hash_key,"text":text}
                articleDictToFile(dict_for_save, save_path_article)
                del text
        else :
            valid = False
            display_text = display_text+"Not valid : Text empty)"
    else :
        valid = False
        display_text = display_text+"Not valid : Dict could not be read)"
    if valid:
        if display :
            print(display_text+"Valid Article)")
        return out_dict
    else :
        if display :
            print(display_text)
        out_dict["valid"] = False
        return out_dict
        

def readArticleFileTable(index_from=0,index_to=1000,save_articles=True,save_final=True,save_steps=False,display_df=True,step_pct=0.1,add_nlp=False):
    stat_field_selection = ["url", "pk", "hash_key", "title", "authors", "publish_date", "keywords_list","summary", "text_len","valid"]# + ["tb.sent", "tb.noun", "tb.word", "tb.char", "tb.pol", "tb.sub", "tb.polaj", "tb.pos", "tb.neg", "vs.pos", "vs.neu", "vs.neg","vs.comp","ts.pos","ts.neg"]
    df = pd.DataFrame([], columns = stat_field_selection)
    df_input = openDFcsv(open_path, filename_input)
    df_input_url = df_input["link"][index_from:min(index_to,df_input.shape[0])]
    art_count = 0
    for url_entry in df_input_url:
        ar_list = readStatsFromURL(url_entry,save_articles,display,art_count,add_nlp)
        df = addDictToDF(df,ar_list)
        art_count = art_count + 1
        if (art_count%int((index_to-index_from)*step_pct) == 0 or art_count==len(df_input_url)) and save_steps :
            df_out = df[output_fields]
            saveDFcsv(df_out, save_path, filename_out+"_"+str(art_count)) # , mode="w"
    df = df[output_fields]
    if save_final :
        saveDFcsv(df, save_path, filename_out+"_final") # , mode="w"
    if display_df :
        display(df)
    return df

def fillDFwithListList(df, ar_list):
    out_list = []
    for i in range(len(ar_list)) :
        out_list.append(ar_list[i])
    # print(len(out_list))    
    df.loc[len(df)] = out_list
    return df

def addDictToDF(df, ar_dict):
    df_add = pd.DataFrame([ar_dict], columns = ar_dict.keys())
    df = pd.concat([df,df_add]).reset_index(drop=True)
    return df

def articleDictToFile(ar_dict, path) :  #requires pk & text
    length_limit = 160
    di_keys = ar_dict.keys()
    if "text" in di_keys and "hash_key" in di_keys :
        if len(ar_dict["hash_key"])<length_limit :
            # cwd = os.getcwd()  # Get the current working directory (cwd)
            # files = os.listdir(cwd)
            saveSTRtxt(ar_dict["text"],path,ar_dict["hash_key"])
            return True
        else :
            print("WARNING : hash_key is too long (longer than " + str(length_limit) + " caracters)")
            return False
    else :
        print("ERROR : articleDictToFile could not find 'text' or 'hash_key' in the dict provided")
        return False

def loadFromFolder(folder_path="", save=False):
    if folder_path == "" :
        folder_path = save_path
    df_out = getStandardDfOutput()
    file_list = os.listdir(folder_path)
    for filename_entry in file_list:
        if ".csv" in filename_entry :
            df = utils.openDFcsv(folder_path,filename_entry.replace(".csv",""))
            df_out = pd.concat([df_out, df], ignore_index=True)
    df_out = df_out.drop_duplicates(subset=['hash_key'])
    if save :
        saveDFcsv(df_out, folder_path, filename_out+"_all_aggregated")
    return df_out

def generateNLPonKeywords(df_main,index_from=0,index_to=500,display_log=True):
    if index_to == -1 :
        index_to = df_main.shape[0]
    df_np = df_main["word_combined_all"].apply(np.array).to_numpy()[0:index_to]
    df_hash = df_main["hash_key"].apply(np.array).to_numpy()[0:index_to]
    mat_index = []
    for i in range(index_from,index_to) :
        if display_log :
            print(" - Generate NLP for article #"+str(i)+"  (char:"+str(len(df_np[i]))+")")
        nlp_dict = ts.analyseText2(str(df_np[i]),True)
        mat_index.append(nlp_dict|{"hash_key":df_hash[i]})
    df = pd.DataFrame(mat_index, columns = list(mat_index[0].keys())) 
    df.set_index("hash_key", inplace=True)
    df_main.set_index("hash_key", inplace=True)
    display(df)
    display(df_main)
    df = df_main.join(df,on='hash_key', how="inner",lsuffix='_k')
    return df

## Variables

In [9]:
env = "arc/"

open_path = "C:/Users/User/OneDrive/Desktop/article/files_3/1_1_query_main/"+env
filename_input = "main_scrap"#"query_large_2010_to_2023"#"query_main_final"

save_path = "C:/Users/User/OneDrive/Desktop/article/files_3/1_2_scarp_main/"+env #parssing_df_main/
filename_out = "test_vol___________XXXXXXXX"

save_path_article = "C:/Users/User/OneDrive/Desktop/article/files_3/1_3_article_main/"+env #article_download_main_2
defdatetime = datetime.now()
input_fields = ["title","link", "published", "source_url", "source_title", "category", "year", "year_month","pk","url_list","url_TLD", "hash_key"]
# output_fields = ["url", "pk","hash_key", "title", "authors", "publish_date", "keywords_list", "text_len","valid"]+ ["tb.sentences", "tb.noun_phrases", "tb.words", "tb.polarity", "tb.subjectivity", "tb.p_pos", "tb.p_neg", "vs.neg", "vs.neu", "vs.pos", "vs.compound"] # 
output_fields = ["url", "pk", "hash_key", "title", "authors", "publish_date", "keywords_list", "summary", "text_len","valid"]# + ["tb.sent", "tb.noun", "tb.word", "tb.char", "tb.pol", "tb.sub", "tb.polaj", "tb.pos", "tb.neg", "vs.pos", "vs.neu", "vs.neg","vs.comp","ts.pos","ts.neg"]

# Main

In [10]:
df = readArticleFileTable(index_from=0,index_to=100,save_final=True,save_steps=True,step_pct=0.01,add_nlp=False,save_articles=True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/User/OneDrive/Desktop/article/files_3/1_1_query_main/arc/main_scrap.csv'

## Init NLP from keywords

## --- BIN ---

In [ ]:
df_out = loadFromFolder()

file_list = os.listdir("C:/Users/User/code_article/parssing_df_main/")
print(file_list)
with open("C:/Users/User/code_article/lol_test.txt") as file: # , "w", encoding="utf-8"
    file.write("test")
    print(str(openSTRtxt("C:/Users/User/code_article/","lol_test")))
article.parse().nlp()
print(article.keywords)
print(article.summary)

import newspaper
from newspaper import Article
from newspaper import fulltext

url='https://news.google.com/topics/'
paper = newspaper.build(url, language="en", memoize_articles=False)

#extracting news category
for category in paper.category_urls():
    print(category)
    
try:
    print(1/0)
except ZeroDivisionError:
    print("could not parse :num")
    pass
print("lol")

import newspaper
cnn_paper = newspaper.build('https://edition.cnn.com/')
for feed_url in cnn_paper.feed_urls():
    print(feed_url)

import newspaper
cnn_paper = newspaper.build('https://edition.cnn.com/')
for article in cnn_paper.articles:
    print(article.url)
    
article.parse()
print(article.html)

art_dict = urlToDict(url,2) #articleToDict(article)
print(art_dict)

start_list = readArticleList(URL_LIST)
os.system('clear')
for start in start_list :
    print(start)
    print("\n--------------------\n")
    

    
readStatsFromURL(url)



article.parse()

#print(article.html)
## --> '<!DOCTYPE HTML><html itemscope itemtype="http://...'

print(article.title)
## --> 'New Year, new laws: Obamacare, pot, guns and drones'

print(article.authors)
## --> ['Leigh Ann Caldwell', 'John Honway']

print(article.publish_date)
## --> datetime.datetime(2013, 12, 30, 0, 0)

print(article.text)
## --> 'Washington (CNN) -- Not everyone subscribes to a New Year's resolution...'

print(article.top_image)
## --> 'http://someCDN.com/blah/blah/blah/file.png'

print(article.images)
## --> ['url_to_img_1', 'url_to_img_2', 'url_to_img_3', ...]

print(article.movies)
## --> ['http://youtube.com/path/to/link.com', ...]



save_path = "C:/Users/User/OneDrive/Desktop/article/files/"
filename = "article"
df = openDFcsv(save_path, filename)
print(len(list(df["link"])))
print(list(df["link"]))

test = df["nlp"].tolist()[0]
print(test)
print(type(test))

dict_test = {"test2":"field2","test1":"field1"}
df = pd.DataFrame([dict_test], columns = ["test1", "test2"])
display(df)
dict_test2 = {"test2":"field25","test5":"field15"}
df2 = pd.DataFrame([dict_test2], columns = ["test1", "test2"])
# df = df.append({"test2":"field25","test1":"field15"})
# df = df.append(pd.Series(dict_test2, index=df.columns))
df3 = pd.concat([df,df2]).reset_index(drop=True)
display(df3)

df['new_date'] = pd.to_datetime(df['publish_date'], errors='coerce')
display(df)

dict_test = {"test2":"field2","test1":"field1"}
dict_test2 = {"test2222":"field25","test1111":"field15"}
print(dict_test | dict_test2)

df1 = utils.openDFcsv(open_path,filename_input).set_index('hash_key')
print(df1.columns)
print(df1.shape)
print(df1.index.value_counts())
df2 = utils.openDFcsv(save_path,filename_out).set_index('hash_key')
print(df2.columns)
print(df2.shape)
print(df2.index.value_counts())
df = df1.join(df2, how="right",on='hash_key', lsuffix='_scrapping', rsuffix='_querieing')
print(df.columns)
display(df)


'title'
'title_detail'
    'type'
    'language'
    'base'
    'value'
'link'
'links' (list)
    'rel'
    'type'
    'href'
'id'
'guidislink'
'published'
'published_parsed'
'summary'
'summary_detail'
    'type'
    'language'
    'base'
    'value'
'source'
    'href'
    'title'
'sub_articles' (list)